## Importing required libraries

In [ ]:
import sys, os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model
import h5py
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import classification_report
import seaborn as sb
import tensorflow as tf

## Loading dataset and removing emotions

In [ ]:
data = pd.read_csv("../input/ck-plus-shuffled/ck_plus.csv")

data = data[data['emotion'] != 1]
data = data[data['emotion'] != 2]

data.loc[data.emotion == 3, 'emotion'] = 1
data.loc[data.emotion == 4, 'emotion'] = 2
data.loc[data.emotion == 5, 'emotion'] = 3
data.loc[data.emotion == 6, 'emotion'] = 4
data.loc[data.emotion == 7, 'emotion'] = 5

## Defining values reused throughout the code

In [ ]:
num_features = 64
num_labels = 6
batch_size = 64
epochs = 100
width, height = 48, 48

## Setting output path and name of model file

In [ ]:
BASEPATH = '../output/kaggle/working'
MODELPATH = 'model_ck_plus.h5'

## Preparing images for analysis (includes normalization)

In [ ]:
pixels = data['pixels'].tolist()
faces = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(width, height)
    faces.append(face.astype('float32')/255.0)

faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)

emotions = pd.get_dummies(data['emotion']).values

## Defining labels which represent each emotion

In [ ]:
labels = {0: 'anger', 1: 'fear', 2: 'happiness', 3: 'neutrality', 4: 'sadness', 5: 'surprise'}

## Printing sample images from the dataset

In [ ]:
fig, axs = plt.subplots(3, 5, figsize=(25, 12))
axs = axs.ravel()
for i in range(15):
    axs[i].imshow(faces[i][:,:,0], cmap='gray')
    axs[i].set_title(str(labels[np.argmax(emotions[i])]))

## Splitting in to train, test, and validation datasets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(faces, emotions, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

## Defining the model

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(num_labels, activation='softmax'))

## Printing model summary

In [ ]:
model.summary()

## Compiling the model and defining options for the model

In [ ]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3, verbose=1)

In [ ]:
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1, mode='auto')

In [ ]:
checkpointer = ModelCheckpoint(MODELPATH, monitor='val_loss', verbose=1, save_best_only=True)

In [ ]:
tensorboard = TensorBoard(log_dir='./logs')

## Training the model

In [ ]:
history = model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)),
          shuffle=True, callbacks=[lr_reducer, tensorboard, early_stopper, checkpointer])

## Defining function for plotting the accuracy and loss of the model over the epochs

#### Taken from https://www.kaggle.com/danbrice/keras-plot-history-full-report-and-grid-search

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    print(len(loss_list) , len(val_loss_list) , len(acc_list) , len(val_acc_list))
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

## Calling the above defined plot_history function

In [ ]:
plot_history(history)

## Evaluating the performance of the model

In [ ]:
scores = model.evaluate(np.array(X_test), np.array(y_test), batch_size=batch_size)
print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))

In [ ]:
predictions=[np.argmax(im) for im in model.predict(np.array(X_test))]

In [ ]:
emotion_labels = [np.argmax(i) for i in np.array(y_test)]

In [ ]:
from sklearn.metrics import accuracy_score

## Plotting the confusion matrix

In [ ]:
em_names = ['fear','surprise','sadness', 'neutrality', 'happiness', 'anger']
fig, ax = plt.subplots(figsize=(8, 8))
sb.heatmap(ax=ax, 
           data=pd.DataFrame(tf.math.confusion_matrix(emotion_labels, predictions, num_classes=6).numpy().astype(int), 
                     index=em_names, 
                     columns=em_names
           ), 
           annot=True, 
           annot_kws={"size": 10},
           fmt='g'
          )

## Printing the classification report

In [ ]:
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = np.argmax(model.predict(X_test), axis=-1)
print(classification_report(Y_test, y_pred, target_names= em_names))

## Plotting sample images misclassified by the model

In [ ]:
rows = 0
cols = 0
disagree = []
for i in range(len(emotion_labels)):
    if emotion_labels[i] != predictions[i]:
        disagree.append(i)
        if len(disagree) == 10:
            break

fig, axs = plt.subplots(2, 5, figsize=(20,9))

for i in range(10):
    image=(np.array(list(X_test)[disagree[i]])/256)[:,:,0]
    axs[cols, rows].imshow(image, cmap='gray')
    axs[cols, rows].set_title('Predicted: ' + str(labels[predictions[disagree[i]]]) + '\nGround Truth: ' + str(labels[emotion_labels[disagree[i]]]))
    rows = rows+1
    if rows == 5:
        cols = cols + 1
        rows = 0

# Evaluating the performance of VGG16 on the same dataset

## Normalizing and stacking the grayscale images into 3 bands as VGG requires 3 band images

In [ ]:
from keras.applications.vgg16 import VGG16

faces = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(width, height)
    face = cv2.merge((face,face,face))
    faces.append(face.astype('float32')/255.0)

faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)

## Splitting in to train, test, and validation datasets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(faces, emotions, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

## Importing the VGG16 model

In [ ]:
model_base = VGG16(include_top=False,
                      weights='imagenet',
                      input_shape=(width, height, 3))
for layer in model_base.layers[:-2]:
    layer.trainable = False

## Defining the model using the VGG16 as the base and adding the Softmax layer

In [ ]:
model = Sequential()
model.add(model_base)
model.add(Flatten())
model.add(Dense(num_labels, activation='softmax'))
model.summary()


## Compiling, training, and evaluating the VGG16 model

In [ ]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])
model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)),
          shuffle=True, callbacks=[lr_reducer, tensorboard, early_stopper, checkpointer])

scores = model.evaluate(np.array(X_test), np.array(y_test), batch_size=batch_size)
print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))